In [4]:
import os
import pandas as pd
import re
import json
from itertools import combinations

# cambiar url para leer los datos de otro subfolder con path relativo
# os.chdir('../')

# Importar stop words en espanol
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))

In [ ]:
# Diccionario de siglas a eliminar dentro del nombre del cliente
# Se ejecuta solo una vez solo si se requiere actualizar la lista de siglas a elimnar

# lista_companias_filtro = ["CORP",'PLC','EPS','CI','SCA','INC','P.A.C','GROUP','SA','SAS','LTDA','LTD','LIMITED','ESP','SOCIEDAD','LP','BV','BIC']
# diccionario = {}
# diccionario["siglas"] = [word.lower() for word in lista_companias_filtro]

# with open("data/archivos_auxiliares/diccionario_siglas.json","w") as f:
#     json.dump(diccionario, f)

In [5]:
# importacion de datos
noticias = pd.read_csv('../datos/noticias.csv')
clientes = pd.read_csv('../datos/clientes.csv')
clientes_noticias = pd.read_csv('../datos/clientes_noticias.csv')
with open("data/archivos_auxiliares/diccionario_siglas.json", 'r') as f:
    diccionario_siglas = json.load(f)
    lista_siglas = diccionario_siglas['siglas']

In [11]:
# funciones utiles
from distutils.command.clean import clean

def check_cliente(cliente, noticia):

    cliente_edit = cliente.split(" ")

    if len(cliente_edit)<3:

        condicion = bool(re.findall(cliente,noticia))
        if condicion:
            return 'Cliente'
        else:
            return 'No Aplica'        
    else:
        combs = [ " ".join(list(x)) for x in combinations(cliente_edit, 2)]
        condicion = [bool(re.findall(x,noticia)) for x in combs]

        if True in condicion:
            return "Cliente_"
            
        else:
            return "no"

def clean_name(text,lista_filtro):

    """
    Esta funcion limpia una cadena de texto excluendo todas aquellas palabras que aparecen dentro de lista_filtro.
    Se usa por defecto la separación de la cadena con espacios en blanco.

    Inputs: 

        text : str

            Texto a limpiar
        
        lista_filtro : list

            Lista de palabras a remover
        
    Outputs:

        String con la cadena de texto limpia

    """
    lista_nombre = text.split(' ')
    nombre_final = [word for word in lista_nombre if word not in lista_filtro and word not in spanish_stop_words]
    return ' '.join(nombre_final)

def elimina_letras_sueltas(text):
    
    text_clean = text.split(" ")
    text_clean = [word for word in text_clean if len(word)>1]

    return " ".join(text_clean)


def clean_df(df, *args, **kwards):

    df_clean = df.copy()
    df_clean['nombre_clean'] = df_clean['nombre'].apply(lambda x: x.replace('.',''))
    df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(lambda x: clean_name(text = x.lower(),*args, **kwards))
    df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(elimina_letras_sueltas)
    df_clean['nombre_clean'] = df_clean['nombre_clean'].str.title()
    df_clean['long_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x))
    df_clean['palabras_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x.split(' ')))
    df_clean = df_clean[['nit',"nombre_clean"]]

    return df_clean

def preprocessing_noticas_clientes(x):

    x['Participacion'] = x.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['news_text_content']))    
    return x

def noticia_cliente_detalle(noticias,clientes_noticias):
    final_df = clientes_noticias.merge(noticias, on = 'news_id', how='left')
    final_df = final_df[['nit','news_id', 'news_text_content']]
    return final_df

In [12]:
# Limpieza de dfs
clientes_clean = clean_df(clientes, lista_filtro = lista_siglas)
noticia_cliente_detalle_df = noticia_cliente_detalle(noticias,clientes_noticias)
participacion_df = clientes_clean.merge(noticia_cliente_detalle_df, on = 'nit', how = 'left')

# Quitar stopr words de noticas para buscar nombre en mayusc de la empresa
participacion_df['cuerpo_not_clean'] = participacion_df['news_text_content'].apply(lambda x: clean_name(text = x, lista_filtro=[]))
participacion_df["Participacion"] = participacion_df.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['cuerpo_not_clean']), axis=1)    

In [13]:
participacion_df['Participacion'].value_counts()

No Aplica    39616
no           30567
Cliente_      2268
Cliente       2166
Name: Participacion, dtype: int64

In [23]:
participacion_df[participacion_df['Participacion']=='no']

,nit,nombre_clean,news_id,news_text_content,cuerpo_not_clean,Participacion
0,805027024,Supermercado Gran Colombia,news12584,"En marzo de 2020, la pandemia encontro a Marco...","En marzo 2020, pandemia encontro Marcos Acuna ...",no
1,805027024,Supermercado Gran Colombia,news12758,Olimpica S.A. informo que en alianza con Plan ...,Olimpica S.A. informo alianza Plan B Investmen...,no
2,805027024,Supermercado Gran Colombia,news13241,La cadena de supermercados estadounidense The ...,La cadena supermercados estadounidense The Fre...,no
3,805027024,Supermercado Gran Colombia,news15048,"'Riqueza Natural', el primer programa de gran ...","'Riqueza Natural', primer programa gran escala...",no
4,805027024,Supermercado Gran Colombia,news15611,"El 2 de julio ultimo, antes de que Cristina Fe...","El 2 julio ultimo, Cristina Fernandez Kirchner...",no
...,...,...,...,...,...,...
74563,830130106,Soenergy International Colombia,news95951,El sector de hidrocarburos en Colombia ha reci...,El sector hidrocarburos Colombia recibido noti...,no
74564,830130106,Soenergy International Colombia,news95953,2 y 0 2 y 0 7 y 1 7 y 1 4 y 5 4 y 5 9 y 8 9 y ...,2 0 2 0 7 1 7 1 4 5 4 5 9 8 9 8 aplica aplica ...,no
74565,830130106,Soenergy International Colombia,news99991,"Durante los anos 60 y 70 del siglo pasado, soc...","Durante anos 60 70 siglo pasado, sociologos ec...",no
74566,830130106,Soenergy International Colombia,news99996,El comportamiento de la economia en los ultimo...,El comportamiento economia ultimos meses hace ...,no


In [24]:
noticias[noticias['news_id']=='news95953'].reset_index().loc[0,"news_url_absolute"]

'https://www.elcolombiano.com/negocios/colombia-no-firmara-nuevos-contratos-de-gas-natural-FN18373886'

In [159]:
clientes['nombre_editado'] = clientes['nombre'].str.lower()
clientes['val_nombre_editado'] = clientes['nombre_editado'].apply(lambda x: bool(re.findall('davivienda',x)))
clientes[clientes['val_nombre_editado']]
noticias['nombre_editado'] = noticias['news_text_content'].str.lower()
noticias['val_nombre_editado'] = noticias['nombre_editado'].apply(lambda x: bool(re.findall('eps',x)))
davi = noticias[noticias['val_nombre_editado']].copy()
davi = davi.merge(clientes_noticias, on = 'news_id', how = 'left')
davi.loc[1,'news_text_content']
clientes[clientes['nit'].isin(list(davi['nit'].unique()))]
x = davi[davi['nit']==805027024].copy().reset_index()
x.loc[0,'news_url_absolute_x'] # news_url_absolute_x, news_title, news_text_content	